# kokudo-suuchi-jouhouの使用例

In [1]:
# import時はksjと書いてください
import ksj

## 国土数値情報の概要情報取得

国土数値情報APIで取得できるデータの一覧と、その識別子（identifier）やareaTypeなどの情報を確認できます。

In [2]:
# 国土数値情報APIでダウンロードできるファイル概要
ksj_summary = ksj.get_summary()
ksj_summary.head()

,identifier,title,field1,field2,areaType
0,A03,三大都市圏計画区域,政策区域,大都市圏,2
1,A09,都市地域,国土（水・土地）,土地利用,3
2,A10,自然公園地域,地域,保護保全,3
3,A11,自然保全地域,地域,保護保全,3
4,A12,農業地域,国土（水・土地）,土地利用,3


## 国土数値情報のURL情報取得

国土数値情報のshapefileが入ったzipファイルのURLを取得することができます。

パラメータの意味についての詳細は[APIのドキュメント](http://nlftp.mlit.go.jp/ksj/api/specification_api_ksj.pdf)でご確認ください。


```
get_url(identifier: str, pref_code=None, mesh_code=None,
            metro_area=None, fiscal_year=None) -> pd.DataFrame:
Parameters
----------
identifier : str
    ファイルの識別子（例：「地価公示」なら'L01'）
pref_code : str, int
    都道府県コード（例：13, '11,12', '11-14'）
mesh_code : str, int
    メッシュコード。areaType=4の場合のみ有効。（例：5340）
metro_area : str, int
    都市圏コード。areaType=2の場合のみ有効。（例：100）
fiscal_year : str, int
    年度。（例：2017, '2015,2019', '2014-2016'）

Returns
-------
pandas.DataFrame
    データの情報とzipファイルのURLが入ったデータフレーム
```

In [3]:
# 国土数値情報のzipファイルのURLを取得（return: pd.DataFrame）
urls = ksj.get_url(identifier="N03", pref_code='11-14', fiscal_year=2019)
urls.head()

,identifier,title,field,year,areaType,areaCode,datum,zipFileUrl,zipFileSize
0,N03,行政区域,政策区域,2019,3,11,1,http://nlftp.mlit.go.jp/ksj/gml/data/N03/N03-2...,3.54MB
1,N03,行政区域,政策区域,2019,3,12,1,http://nlftp.mlit.go.jp/ksj/gml/data/N03/N03-2...,6.17MB
2,N03,行政区域,政策区域,2019,3,13,1,http://nlftp.mlit.go.jp/ksj/gml/data/N03/N03-2...,12.20MB
3,N03,行政区域,政策区域,2019,3,14,1,http://nlftp.mlit.go.jp/ksj/gml/data/N03/N03-2...,5.22MB


## ファイルの読み込み

`ksj.get_shp()`にURLとファイルの保存先を指定することでシェープファイルをダウンロードします。

`ksj.read_shp()`にURLを渡した場合、ダウンロードしたあと解凍してgeopandasで読み込みます。


In [8]:
# ファイルを一時フォルダにダウンロードし、解凍してgeopandasで読み込む
url = urls["zipFileUrl"][0]
shape_gdf = ksj.read_shp(url)
shape_gdf.head()

,N03_001,N03_002,N03_003,N03_004,N03_007,geometry
0,埼玉県,None,さいたま市,西区,11101,"POLYGON ((139.54776 35.93420, 139.54720 35.934..."
1,埼玉県,None,さいたま市,西区,11101,"POLYGON ((139.54776 35.93420, 139.54804 35.934..."
2,埼玉県,None,さいたま市,北区,11102,"POLYGON ((139.61753 35.96486, 139.61798 35.964..."
3,埼玉県,None,さいたま市,大宮区,11103,"POLYGON ((139.63768 35.92278, 139.63804 35.922..."
4,埼玉県,None,さいたま市,見沼区,11104,"POLYGON ((139.66718 35.96444, 139.66739 35.964..."


## 列名の変換

簡単に自動変換できる列名（年度によって意味が変わることのない列）に限り、列名を日本語に変換します。



In [9]:
# 列名を日本語に変換
shape_gdf = ksj.translate(shape_gdf)
shape_gdf.head()

,都道府県名,支庁名,郡政令都市,市区町村名,行政区域コード,geometry
0,埼玉県,None,さいたま市,西区,11101,"POLYGON ((139.54776 35.93420, 139.54720 35.934..."
1,埼玉県,None,さいたま市,西区,11101,"POLYGON ((139.54776 35.93420, 139.54804 35.934..."
2,埼玉県,None,さいたま市,北区,11102,"POLYGON ((139.61753 35.96486, 139.61798 35.964..."
3,埼玉県,None,さいたま市,大宮区,11103,"POLYGON ((139.63768 35.92278, 139.63804 35.922..."
4,埼玉県,None,さいたま市,見沼区,11104,"POLYGON ((139.66718 35.96444, 139.66739 35.964..."


## 列名対応表のダウンロード

国土数値情報から列名の対応表をダウンロードし、pandasで開きます。

In [6]:
# 列名の対応表をダウンロードして開く
book = ksj.get_column_table()
book.query("識別子 == 'N03'")

,識別子,データ項目,タグ名,対応番号,属性名,
59,N03,行政区域,AdministrativeBoundary,N03_001,都道府県名,NaN
60,N03,行政区域,AdministrativeBoundary,N03_002,支庁名,NaN
61,N03,行政区域,AdministrativeBoundary,N03_003,郡政令都市,NaN
62,N03,行政区域,AdministrativeBoundary,N03_004,市区町村名,NaN
63,N03,行政区域,AdministrativeBoundary,N03_005,成立年月日,NaN
64,N03,行政区域,AdministrativeBoundary,N03_006,消滅年月日,NaN
65,N03,行政区域,AdministrativeBoundary,N03_007,行政区域コード,NaN
